In [0]:
%run "../includes/configuration"

# DRIVER STANDINGS

In [0]:
race_results_df = spark.read.parquet(f"{PRESENTATION_FOLDER_PATH}/race_results")

In [0]:
from pyspark.sql.functions import sum, when, count, col 

driver_standings_df = race_results_df. \
                    groupBy("race_year", "driver_name", "driver_nationality", "constructor_name") \
                    .agg(sum("points").alias("total_points"), 
                        count(when(col("position") == 1, True)).alias("total_wins"))

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import asc, desc, rank 

driver_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("total_wins"))
final_df = driver_standings_df.withColumn("rank", rank().over(driver_rank_spec))

In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.driver_standings")

# CONSTRUCTOR STANDING

In [0]:
from pyspark.sql.functions import sum, when, count, col 

constructor_standings_df = race_results_df. \
                    groupBy("race_year", "constructor_name") \
                    .agg(sum("points").alias("total_points"), 
                        count(when(col("position") == 1, True)).alias("total_wins"))

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import asc, desc, rank 

constructor_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("total_wins"))
final_df = constructor_standings_df.withColumn("rank", rank().over(constructor_rank_spec))

In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.constructor_standings")